In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import datetime as dt

In [2]:
def get_data(ticker,start_date,end_date):
    StockData = pdr.get_data_yahoo(ticker,start=start_date,end=end_date)
    StockPrice = StockData['Adj Close']
    returns = StockPrice.pct_change()
    avg_return = returns.mean()
    CoV_mat = returns.cov()
    return returns, avg_return, CoV_mat

def Portfolio_Performace(weights, avg_return, CoV_mat, trading_days):
    portfolio_returns = np.sum(avg_return*weights)*trading_days
    portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(CoV_mat,weights)))*np.sqrt(trading_days)
    return portfolio_returns, portfolio_risk
    

In [3]:
ticker_list = ['DMART','LAURUSLABS', 'HAVELLS', 'PIIND', 'SBIN']
ticker = [i+'.NS' for i in ticker_list]
end_date = dt.datetime.today()
start_date = end_date - dt.timedelta(days=800)

returns, avg_return, CoV_mat = get_data(ticker,start_date,end_date)
returns.dropna(inplace=True)

weights = np.random.random(len(ticker))
weights /= np.sum(weights)

returns['Portfolio_return'] = returns.dot(weights)

In [4]:
def historical_VaR(returns, alpha=5):
    
    if isinstance(returns, pd.Series):
        return np.percentile(returns,alpha)
    
    elif isinstance(returns, pd.DataFrame):
        return returns.agg(historical_VaR,alpha=alpha)
    
    else:
        raise TypeError("Expexted returns to be DataFrame or Series")
        
def historical_Cond_VaR(returns, alpha=5):
    
    if isinstance(returns, pd.Series):
        below_VaR = returns <= historical_VaR(returns,alpha=alpha)
        return returns[below_VaR].mean()
    
    elif isinstance(returns, pd.DataFrame):
        return returns.agg(historical_Cond_VaR,alpha=alpha)
    
    else:
        raise TypeError("Expexted returns to be DataFrame or Series")

In [5]:
trading_days = 1 #days
Initial_investment = 100000

VaR = -historical_VaR(returns.Portfolio_return,alpha=5)
CVaR = -historical_Cond_VaR(returns.Portfolio_return,alpha=5)
portfolio_returns, portfolio_risk = Portfolio_Performace(weights, avg_return, CoV_mat, trading_days)

print(f"Expected return of Portfolio in {trading_days} days: {np.round(portfolio_returns*Initial_investment,2)}")
print(f"Value at Risk 95th CI: {np.round(VaR*Initial_investment,2)}")
print(f"Conditional VaR 95th CI: {np.round(CVaR*Initial_investment,2)}")

Expected return of Portfolio in 1 days: 205.59
Value at Risk 95th CI: 2229.69
Conditional VaR 95th CI: 3050.98
